# Lab 3.2: Forward Propagation Vectorization

## Learning Objectives
- Implement forward propagation using vectorized operations
- Understand the mathematics behind forward propagation
- Optimize neural network computations using NumPy
- Handle multiple training examples simultaneously

## Duration: 45 minutes

## Prerequisites
- Completion of Lab 3.1
- Understanding of matrix operations
- Basic knowledge of activation functions

## Setup and Environment

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_classification, make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.style.use('seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else 'default')
plt.rcParams['figure.figsize'] = (12, 8)

print("Environment setup complete!")
print(f"NumPy version: {np.__version__}")

## Part 1: Activation Functions (10 minutes)

First, let's implement the activation functions we'll need for forward propagation:

In [ ]:
class ActivationFunctions:
    """
    Collection of activation functions and their derivatives
    """
    
    @staticmethod
    def relu(z):
        """ReLU activation function"""
        return np.maximum(0, z)
    
    @staticmethod
    def relu_derivative(z):
        """Derivative of ReLU function"""
        return (z > 0).astype(float)
    
    @staticmethod
    def sigmoid(z):
        """Sigmoid activation function"""
        # Clip z to prevent overflow
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))
    
    @staticmethod
    def sigmoid_derivative(z):
        """Derivative of sigmoid function"""
        s = ActivationFunctions.sigmoid(z)
        return s * (1 - s)
    
    @staticmethod
    def tanh(z):
        """Hyperbolic tangent activation function"""
        return np.tanh(z)
    
    @staticmethod
    def tanh_derivative(z):
        """Derivative of tanh function"""
        return 1 - np.tanh(z)**2
    
    @staticmethod
    def linear(z):
        """Linear activation function (identity)"""
        return z
    
    @staticmethod
    def linear_derivative(z):
        """Derivative of linear function"""
        return np.ones_like(z)
    
    @staticmethod
    def softmax(z):
        """Softmax activation function for multi-class classification"""
        # Subtract max for numerical stability
        exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))
        return exp_z / np.sum(exp_z, axis=0, keepdims=True)

# Test activation functions
test_values = np.array([-2, -1, 0, 1, 2])
print("Testing activation functions:")
print(f"Input: {test_values}")
print(f"ReLU: {ActivationFunctions.relu(test_values)}")
print(f"Sigmoid: {ActivationFunctions.sigmoid(test_values)}")
print(f"Tanh: {ActivationFunctions.tanh(test_values)}")
print("\nActivation functions implemented successfully!")

## Part 2: Forward Propagation Implementation (15 minutes)

### Understanding Forward Propagation Mathematics

For each layer l:
- Linear transformation: Z^[l] = W^[l] * A^[l-1] + b^[l]
- Activation: A^[l] = g^[l](Z^[l])

Where:
- W^[l]: Weight matrix for layer l
- b^[l]: Bias vector for layer l
- A^[l-1]: Activations from previous layer
- g^[l]: Activation function for layer l

In [ ]:
class ForwardPropagation:
    """
    Vectorized forward propagation implementation
    """
    
    def __init__(self):
        self.activations = ActivationFunctions()
        self.cache = {}  # Store intermediate values for backpropagation
    
    def initialize_parameters(self, layer_dims):
        """
        Initialize weights and biases for all layers
        
        Parameters:
        layer_dims: list of layer dimensions [input_size, hidden1, hidden2, ..., output_size]
        
        Returns:
        parameters: dictionary containing weights and biases
        """
        parameters = {}
        
        for l in range(1, len(layer_dims)):
            # Xavier/Glorot initialization
            parameters[f'W{l}'] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(2.0 / layer_dims[l-1])
            parameters[f'b{l}'] = np.zeros((layer_dims[l], 1))
        
        return parameters
    
    def linear_forward(self, A_prev, W, b):
        """
        Implement the linear part of forward propagation
        
        Parameters:
        A_prev: activations from previous layer (n_prev, m)
        W: weights matrix (n_current, n_prev)
        b: bias vector (n_current, 1)
        
        Returns:
        Z: linear transformation result
        cache: tuple containing (A_prev, W, b) for backpropagation
        """
        Z = np.dot(W, A_prev) + b
        cache = (A_prev, W, b)
        
        return Z, cache
    
    def linear_activation_forward(self, A_prev, W, b, activation):
        """
        Implement forward propagation for one layer (linear + activation)
        
        Parameters:
        A_prev: activations from previous layer
        W: weights matrix
        b: bias vector
        activation: activation function name
        
        Returns:
        A: post-activation values
        cache: tuple containing linear_cache and activation_cache
        """
        # Linear forward
        Z, linear_cache = self.linear_forward(A_prev, W, b)
        
        # Activation forward
        if activation == 'sigmoid':
            A = self.activations.sigmoid(Z)
        elif activation == 'relu':
            A = self.activations.relu(Z)
        elif activation == 'tanh':
            A = self.activations.tanh(Z)
        elif activation == 'linear':
            A = self.activations.linear(Z)
        elif activation == 'softmax':
            A = self.activations.softmax(Z)
        else:
            raise ValueError(f"Unsupported activation function: {activation}")
        
        cache = (linear_cache, Z)  # Store both linear cache and Z for backprop
        
        return A, cache
    
    def forward_propagation(self, X, parameters, activation_functions):
        """
        Implement forward propagation for entire network
        
        Parameters:
        X: input data (input_size, m)
        parameters: dictionary containing weights and biases
        activation_functions: list of activation functions for each layer
        
        Returns:
        AL: final layer activations (predictions)
        caches: list of caches for each layer
        """
        caches = []
        A = X
        
        # Number of layers (excluding input layer)
        L = len(parameters) // 2
        
        # Forward propagation through all layers
        for l in range(1, L + 1):
            A_prev = A
            W = parameters[f'W{l}']
            b = parameters[f'b{l}']
            activation = activation_functions[l-1]
            
            A, cache = self.linear_activation_forward(A_prev, W, b, activation)
            caches.append(cache)
        
        return A, caches
    
    def predict(self, X, parameters, activation_functions, threshold=0.5):
        """
        Make predictions using the trained network
        
        Parameters:
        X: input data
        parameters: network parameters
        activation_functions: activation functions
        threshold: decision threshold for binary classification
        
        Returns:
        predictions: predicted labels
        probabilities: prediction probabilities
        """
        probabilities, _ = self.forward_propagation(X, parameters, activation_functions)
        
        if probabilities.shape[0] == 1:  # Binary classification
            predictions = (probabilities > threshold).astype(int)
        else:  # Multi-class classification
            predictions = np.argmax(probabilities, axis=0)
        
        return predictions, probabilities

print("ForwardPropagation class implemented successfully!")

## Part 3: Testing Forward Propagation (10 minutes)

### Test 1: Simple Network

In [ ]:
# Create a simple test network
forward_prop = ForwardPropagation()

# Network architecture: 3 inputs -> 4 hidden (ReLU) -> 1 output (sigmoid)
layer_dims = [3, 4, 1]
activation_functions = ['relu', 'sigmoid']

# Initialize parameters
parameters = forward_prop.initialize_parameters(layer_dims)

print("Network Architecture:")
print(f"Layer dimensions: {layer_dims}")
print(f"Activation functions: {activation_functions}")
print("\nInitialized Parameters:")
for key, value in parameters.items():
    print(f"{key}: shape {value.shape}")

# Create test input
m = 5  # Number of examples
X = np.random.randn(3, m)  # 3 features, 5 examples

print(f"\nInput shape: {X.shape}")
print(f"Input data (first 3 examples):\n{X[:, :3]}")

In [ ]:
# Perform forward propagation
predictions, caches = forward_prop.forward_propagation(X, parameters, activation_functions)

print("Forward Propagation Results:")
print(f"Predictions shape: {predictions.shape}")
print(f"Predictions: {predictions.flatten()}")
print(f"Number of cached layers: {len(caches)}")

# Test predictions
pred_labels, pred_probs = forward_prop.predict(X, parameters, activation_functions)
print(f"\nPredicted labels: {pred_labels.flatten()}")
print(f"Predicted probabilities: {pred_probs.flatten()}")

### Test 2: Multi-class Classification Network

In [ ]:
# Create a multi-class classification network
# Architecture: 2 inputs -> 5 hidden (ReLU) -> 3 outputs (softmax)
multiclass_dims = [2, 5, 3]
multiclass_activations = ['relu', 'softmax']

# Initialize parameters
mc_parameters = forward_prop.initialize_parameters(multiclass_dims)

# Create test data
X_mc = np.random.randn(2, 8)  # 2 features, 8 examples

print("Multi-class Network:")
print(f"Architecture: {multiclass_dims}")
print(f"Activations: {multiclass_activations}")
print(f"Input shape: {X_mc.shape}")

# Forward propagation
mc_predictions, mc_caches = forward_prop.forward_propagation(X_mc, mc_parameters, multiclass_activations)

print(f"\nOutput shape: {mc_predictions.shape}")
print(f"Sum of probabilities (should be ~1.0): {np.sum(mc_predictions, axis=0)}")
print(f"Predictions (first 4 examples):\n{mc_predictions[:, :4]}")

# Make predictions
mc_labels, mc_probs = forward_prop.predict(X_mc, mc_parameters, multiclass_activations)
print(f"\nPredicted classes: {mc_labels}")

## Part 4: Performance Comparison (5 minutes)

Let's compare vectorized vs non-vectorized implementations:

In [ ]:
def forward_propagation_loop(X, parameters, activation_functions):
    """
    Non-vectorized forward propagation (for comparison)
    Processes examples one by one
    """
    m = X.shape[1]
    L = len(parameters) // 2
    
    # Initialize output array
    if activation_functions[-1] == 'softmax':
        n_output = parameters[f'W{L}'].shape[0]
    else:
        n_output = parameters[f'W{L}'].shape[0]
    
    AL_loop = np.zeros((n_output, m))
    
    # Process each example individually
    for i in range(m):
        x_i = X[:, i:i+1]  # Single example
        a = x_i
        
        # Forward through all layers
        for l in range(1, L + 1):
            w = parameters[f'W{l}']
            b = parameters[f'b{l}']
            z = np.dot(w, a) + b
            
            # Apply activation
            if activation_functions[l-1] == 'relu':
                a = np.maximum(0, z)
            elif activation_functions[l-1] == 'sigmoid':
                a = 1 / (1 + np.exp(-np.clip(z, -500, 500)))
            elif activation_functions[l-1] == 'softmax':
                exp_z = np.exp(z - np.max(z))
                a = exp_z / np.sum(exp_z)
        
        AL_loop[:, i:i+1] = a
    
    return AL_loop

# Performance comparison
# Create larger test data
large_X = np.random.randn(10, 1000)  # 10 features, 1000 examples
large_dims = [10, 20, 5, 1]
large_activations = ['relu', 'relu', 'sigmoid']
large_params = forward_prop.initialize_parameters(large_dims)

print("Performance Comparison:")
print(f"Test data: {large_X.shape[0]} features, {large_X.shape[1]} examples")
print(f"Network: {large_dims}")

# Vectorized version
start_time = time.time()
vectorized_result, _ = forward_prop.forward_propagation(large_X, large_params, large_activations)
vectorized_time = time.time() - start_time

# Loop version
start_time = time.time()
loop_result = forward_propagation_loop(large_X, large_params, large_activations)
loop_time = time.time() - start_time

print(f"\nVectorized time: {vectorized_time:.4f} seconds")
print(f"Loop time: {loop_time:.4f} seconds")
print(f"Speedup: {loop_time/vectorized_time:.1f}x faster")

# Verify results are the same
diff = np.max(np.abs(vectorized_result - loop_result))
print(f"Maximum difference: {diff:.10f}")
print("✅ Results match!" if diff < 1e-10 else "❌ Results don't match!")

## Part 5: Real Data Example (5 minutes)

Let's test our forward propagation on a real dataset:

In [ ]:
# Generate a realistic dataset
X_real, y_real = make_moons(n_samples=200, noise=0.3, random_state=42)

# Prepare data
X_real = X_real.T  # Transpose to (features, examples)
y_real = y_real.reshape(1, -1)  # Reshape to (1, examples)

# Split into train/test
X_train = X_real[:, :150]
X_test = X_real[:, 150:]
y_train = y_real[:, :150]
y_test = y_real[:, 150:]

print("Real Dataset:")
print(f"Training data: {X_train.shape}")
print(f"Training labels: {y_train.shape}")
print(f"Test data: {X_test.shape}")
print(f"Test labels: {y_test.shape}")

# Visualize the dataset
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
colors = ['red', 'blue']
for i in range(2):
    mask = y_train.flatten() == i
    plt.scatter(X_train[0, mask], X_train[1, mask], c=colors[i], label=f'Class {i}', alpha=0.7)
plt.title('Training Data')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
for i in range(2):
    mask = y_test.flatten() == i
    plt.scatter(X_test[0, mask], X_test[1, mask], c=colors[i], label=f'Class {i}', alpha=0.7)
plt.title('Test Data')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Create network for the real dataset
real_dims = [2, 4, 1]  # 2 features -> 4 hidden -> 1 output
real_activations = ['tanh', 'sigmoid']
real_params = forward_prop.initialize_parameters(real_dims)

# Forward propagation on training data
train_predictions, train_caches = forward_prop.forward_propagation(X_train, real_params, real_activations)

# Forward propagation on test data
test_predictions, test_caches = forward_prop.forward_propagation(X_test, real_params, real_activations)

print("Forward Propagation on Real Data:")
print(f"Training predictions shape: {train_predictions.shape}")
print(f"Test predictions shape: {test_predictions.shape}")

# Make binary predictions
train_pred_labels, train_probs = forward_prop.predict(X_train, real_params, real_activations)
test_pred_labels, test_probs = forward_prop.predict(X_test, real_params, real_activations)

print(f"\nTraining prediction summary:")
print(f"Predicted class 0: {np.sum(train_pred_labels == 0)} examples")
print(f"Predicted class 1: {np.sum(train_pred_labels == 1)} examples")
print(f"Actual class 0: {np.sum(y_train == 0)} examples")
print(f"Actual class 1: {np.sum(y_train == 1)} examples")

print(f"\nTest prediction summary:")
print(f"Predicted class 0: {np.sum(test_pred_labels == 0)} examples")
print(f"Predicted class 1: {np.sum(test_pred_labels == 1)} examples")
print(f"Actual class 0: {np.sum(y_test == 0)} examples")
print(f"Actual class 1: {np.sum(y_test == 1)} examples")

# Note: Random weights won't give good accuracy - this is just testing forward propagation
train_accuracy = np.mean(train_pred_labels == y_train) * 100
test_accuracy = np.mean(test_pred_labels == y_test) * 100

print(f"\nRandom weights performance:")
print(f"Training accuracy: {train_accuracy:.1f}%")
print(f"Test accuracy: {test_accuracy:.1f}%")
print("Note: Low accuracy expected with random weights!")

## Progress Tracking Checklist

Check off each item as you complete it:

- [ ] **Environment Setup**: Imported libraries and configured environment
- [ ] **Activation Functions**: Implemented and tested activation functions
- [ ] **Forward Propagation Class**: Created vectorized forward propagation
- [ ] **Parameter Initialization**: Implemented weight and bias initialization
- [ ] **Linear Forward**: Implemented linear transformation step
- [ ] **Activation Forward**: Implemented activation step
- [ ] **Full Forward Pass**: Implemented complete forward propagation
- [ ] **Simple Network Test**: Tested on simple binary classification network
- [ ] **Multi-class Test**: Tested on multi-class classification network
- [ ] **Performance Comparison**: Compared vectorized vs loop implementation
- [ ] **Real Data Test**: Applied forward propagation to real dataset
- [ ] **Lab Completion**: Successfully completed all exercises

## Key Concepts Summary

### What You've Learned:
1. **Vectorization Benefits**: Massive performance improvements over loops
2. **Forward Propagation Steps**: Linear transformation → Activation
3. **Matrix Operations**: Understanding of weight-input multiplication
4. **Batch Processing**: Handling multiple examples simultaneously
5. **Activation Functions**: Different functions for different layer types

### Key Mathematical Concepts:
- **Linear Transformation**: Z^[l] = W^[l] * A^[l-1] + b^[l]
- **Activation**: A^[l] = g^[l](Z^[l])
- **Vectorization**: Processing multiple examples in parallel
- **Parameter Shapes**: Ensuring compatible matrix dimensions

## Validation Steps

Run the following cells to validate your understanding:

In [ ]:
# Validation Test 1: Shape Consistency
def test_shape_consistency():
    """Test if forward propagation maintains correct shapes"""
    try:
        # Test different architectures
        test_cases = [
            {'dims': [3, 5, 1], 'examples': 10},
            {'dims': [10, 20, 15, 5], 'examples': 50},
            {'dims': [2, 8, 3], 'examples': 25}
        ]
        
        for case in test_cases:
            dims = case['dims']
            m = case['examples']
            
            # Create test data
            X_test = np.random.randn(dims[0], m)
            params_test = forward_prop.initialize_parameters(dims)
            activations_test = ['relu'] * (len(dims) - 2) + ['sigmoid']
            
            # Forward propagation
            output, caches = forward_prop.forward_propagation(X_test, params_test, activations_test)
            
            # Check output shape
            expected_shape = (dims[-1], m)
            assert output.shape == expected_shape, f"Expected {expected_shape}, got {output.shape}"
            
            # Check cache length
            expected_layers = len(dims) - 1
            assert len(caches) == expected_layers, f"Expected {expected_layers} caches, got {len(caches)}"
        
        print("✅ Shape consistency test passed!")
        return True
    except Exception as e:
        print(f"❌ Shape consistency test failed: {e}")
        return False

test_shape_consistency()

In [ ]:
# Validation Test 2: Activation Function Correctness
def test_activation_functions():
    """Test activation functions work correctly"""
    try:
        # Test specific values
        z = np.array([[-1, 0, 1], [2, -2, 0.5]])
        
        # ReLU test
        relu_result = ActivationFunctions.relu(z)
        expected_relu = np.array([[0, 0, 1], [2, 0, 0.5]])
        assert np.allclose(relu_result, expected_relu), "ReLU test failed"
        
        # Sigmoid test (check range)
        sigmoid_result = ActivationFunctions.sigmoid(z)
        assert np.all((sigmoid_result >= 0) & (sigmoid_result <= 1)), "Sigmoid range test failed"
        
        # Softmax test (check probabilities sum to 1)
        softmax_result = ActivationFunctions.softmax(z)
        column_sums = np.sum(softmax_result, axis=0)
        assert np.allclose(column_sums, 1.0), "Softmax probability sum test failed"
        
        print("✅ Activation functions test passed!")
        return True
    except Exception as e:
        print(f"❌ Activation functions test failed: {e}")
        return False

test_activation_functions()

In [ ]:
# Validation Test 3: Prediction Consistency
def test_prediction_consistency():
    """Test if predictions are consistent with forward propagation"""
    try:
        # Create test network
        dims = [4, 6, 1]
        X_test = np.random.randn(4, 20)
        params = forward_prop.initialize_parameters(dims)
        activations = ['relu', 'sigmoid']
        
        # Forward propagation
        output, _ = forward_prop.forward_propagation(X_test, params, activations)
        
        # Predictions
        pred_labels, pred_probs = forward_prop.predict(X_test, params, activations)
        
        # Check consistency
        assert np.allclose(output, pred_probs), "Prediction probabilities don't match forward propagation"
        
        # Check binary predictions
        expected_labels = (output > 0.5).astype(int)
        assert np.array_equal(pred_labels, expected_labels), "Prediction labels incorrect"
        
        print("✅ Prediction consistency test passed!")
        return True
    except Exception as e:
        print(f"❌ Prediction consistency test failed: {e}")
        return False

test_prediction_consistency()

## Troubleshooting Guide

### Common Issues and Solutions:

**Issue 1: Shape mismatch errors**
- **Cause**: Incompatible matrix dimensions
- **Solution**: Check that W^[l] has shape (n^[l], n^[l-1]) and A^[l-1] has shape (n^[l-1], m)

**Issue 2: Numerical overflow in sigmoid/softmax**
- **Cause**: Very large input values
- **Solution**: Use np.clip() to limit input range, implemented in our functions

**Issue 3: Softmax probabilities don't sum to 1**
- **Cause**: Numerical instability
- **Solution**: Subtract max value before exponential (implemented)

**Issue 4: Slow performance**
- **Cause**: Using loops instead of vectorization
- **Solution**: Use matrix operations with NumPy, process all examples together

**Issue 5: Cache errors in backpropagation**
- **Cause**: Incorrect cache structure
- **Solution**: Ensure each cache contains (linear_cache, activation_cache)

### Getting Help:
- Check input/output shapes carefully
- Verify activation function implementations
- Test with small examples first
- Print intermediate values for debugging

## Cleanup Instructions

1. **Save your work**: Save this notebook with your implementations
2. **Clear output**: Cell → All Output → Clear (optional, saves space)
3. **Close plots**: Close any open matplotlib windows
4. **Memory cleanup**: Variables will be cleared when kernel is restarted

In [ ]:
# Final summary
print("🎉 Lab 3.2: Forward Propagation Vectorization Completed!")
print("\n📋 What you accomplished:")
print("✅ Implemented vectorized forward propagation")
print("✅ Created efficient activation functions")
print("✅ Tested on multiple network architectures")
print("✅ Demonstrated massive performance improvements")
print("✅ Applied to real datasets")
print("\n🎯 Next: Lab 3.3 - Backward Propagation Implementation")

# Optional: Clean up memory
import gc
gc.collect()
print("\n🧹 Memory cleaned up successfully!")